In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import gradio as gr

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.drop('id',axis=1,inplace=True)

In [4]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [5]:
data_sentences = df['comment_text']
y = df[df.columns[1:]].to_numpy()

In [6]:
y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [7]:
k = max(len(i.split()) for i in data_sentences)
k

1411

In [8]:
vectorizer = tf.keras.layers.TextVectorization(max_tokens=200000,
                                               output_sequence_length=1411,
                                               output_mode='int')

In [9]:
vectorizer.adapt(data_sentences)

In [15]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [16]:
vectorizer("Hi Aron, I hate you")

<tf.Tensor: shape=(1500,), dtype=int64, numpy=array([  171, 98189,     8, ...,     0,     0,     0], dtype=int64)>

In [17]:
vectorized_text = vectorizer(data_sentences)
vectorized_text

<tf.Tensor: shape=(159571, 1500), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [18]:
data = tf.data.Dataset.from_tensor_slices((vectorized_text,y))
data = data.cache()
data = data.shuffle(160000)
data= data.batch(16)
data = data.prefetch(8)

In [19]:
data.as_numpy_iterator().next()

(array([[ 95615,  65325,  95615, ...,      0,      0,      0],
        [   451, 130072,    964, ...,      0,      0,      0],
        [   704,    111,      4, ...,      0,      0,      0],
        ...,
        [     8,     69,     61, ...,      0,      0,      0],
        [   270,    199,      2, ...,      0,      0,      0],
        [    46,     33,     15, ...,      0,      0,      0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [20]:
train_data = data.take(int(len(data)*0.7))
val_data = data.skip(int(len(data)*0.7)).take(int(len(data)*0.2))
test_data = data.skip(int(len(data)*0.9)).take(int(len(data)*0.1))

In [22]:
train_generator = train_data.as_numpy_iterator()

997

In [23]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(1700001,32))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,activation='tanh')))
model.add(tf.keras.layers.Dense(128,activation = 'relu'))
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(128,activation = 'relu'))
model.add(tf.keras.layers.Dense(6,activation='sigmoid'))

In [24]:
model.compile(loss = 'BinaryCrossentropy',optimizer = 'adam')

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          54400032  
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [26]:
mod = model.fit(train_data,epochs=1,validation_data=val_data)

  23/6981 [..............................] - ETA: 5:57:21 - loss: 0.4569

In [10]:
mod = tf.keras.models.load_model("model3.h5")

In [11]:
text = vectorizer("i hate you")
k = (mod.predict(np.expand_dims(text,0))>0.5).astype(int)

1/1 [==============================] - 1s 1s/step


In [12]:
k

array([[1, 0, 0, 0, 0, 0]])

In [16]:
import gradio as gr

In [13]:
def score(commet):
    vec = vectorizer([commet])
    result = mod.predict(vec)

    text = ""
    for i,col in enumerate(df.columns[2:]):
        text+= "{} : {}\n".format(col,result[0][i]>0.5)
    return text

In [14]:
inter = gr.Interface(fn = score,
                     inputs=gr.inputs.Textbox(lines=2,placeholder="aron's face"),
                     outputs="text")

C:\Users\sanji\AppData\Roaming\Python\Python310\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Users\sanji\AppData\Roaming\Python\Python310\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
C:\Users\sanji\AppData\Roaming\Python\Python310\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [15]:
inter.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 113ms/step
